In [10]:
import streamlit as st
import pandas as pd
import numpy as np

import psycopg2
import plotly.express as px

def init_connection():
    return psycopg2.connect(**st.secrets["postgres"])
conn = init_connection()

In [12]:
def load_data(nrows):
    df = pd.read_sql_query('select * from "atendimentos"',con=conn)
    return df

data = load_data(1000)
data.tail()

,id,tipo_senha,numeracao,codigo_senha,data_emissao,guiche,data_atendimento
14,17,SP,1,SP001,2022-09-07 01:23:11.125120,2.0,2022-09-06 22:50:25.045881
15,18,SP,1,SP001,2022-09-07 01:23:13.167156,2.0,2022-09-06 22:50:28.798898
16,19,SP,2,SP002,2022-09-07 01:30:29.742103,2.0,2022-09-06 22:50:35.747990
17,20,SP,3,SP003,2022-09-07 01:30:31.450539,3.0,2022-09-06 22:51:02.753157
18,21,SP,4,SP004,2022-09-07 01:30:33.132797,1.0,2022-09-06 22:51:08.919118


0     22
1     22
2     22
3     22
4     22
5     22
6     22
7     22
8     22
9      0
10     1
11    22
12    22
13     1
14     1
15     1
16     1
17     1
18     1
Name: data_emissao, dtype: int64

In [27]:
data['hora_emissao'] = data['data_emissao'].dt.hour
data['hora_atendimento'] = data['data_atendimento'].dt.hour

In [29]:
data.rename(columns = {"hora_emissao":"Hora Emissao", "id":"Count"}).groupby(by=["Hora Emissao"]).count()[["Count"]].sort_values(by="Count")

,Count
Hora Emissao,
0,1
1,7
22,11


In [25]:
data.groupby(by=["tipo_senha"]).count()[["id"]].sort_values(by="id")

,id
tipo_senha,
SE,5
SP,14


In [24]:
data.groupby('tipo_senha').size()

tipo_senha
SE     5
SP    14
dtype: int64

In [13]:
tipos = data['tipo_senha'].unique()
for tipo in tipos:
    cond1 = ~data["data_atendimento"].isna()
    cond2 = data["tipo_senha"] == tipo
    senha_por_tipo = int(len(data[cond1 & cond2]))
    print(f"Senhas Atendidas {tipo}: {senha_por_tipo}")

Senhas Atendidas SE: 5
Senhas Atendidas SP: 13
